In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [38]:
import numpy as np

In [66]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="anomaly2",dimension=6, metric="cosine",spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [19]:
import http.client
import json

conn = http.client.HTTPSConnection("cnbc.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "5ed523c6cdmsh5a89356e426a93bp18e1afjsn5857db27f5c5",
    'x-rapidapi-host': "cnbc.p.rapidapi.com"
}

conn.request("GET", "/symbols/v2/get-chart?symbol=IT2Y&interval=1Y", headers=headers)

res = conn.getresponse()
data = res.read()
decoded_data = data.decode("utf-8")
json_data = json.loads(decoded_data)

# Print the JSON data
print(json_data)

{'barData': {'priceBars': [{'open': '2.9950', 'high': '3.0400', 'low': '2.9270', 'close': '2.9990', 'volume': 0, 'tradeTime': '20230113000000', 'tradeTimeinMills': 1673586000000}, {'open': '2.9980', 'high': '3.0410', 'low': '2.9980', 'close': '3.0150', 'volume': 0, 'tradeTime': '20230114000000', 'tradeTimeinMills': 1673672400000}, {'open': '3.0150', 'high': '3.0460', 'low': '2.9890', 'close': '3.0030', 'volume': 0, 'tradeTime': '20230116000000', 'tradeTimeinMills': 1673845200000}, {'open': '2.9990', 'high': '3.0590', 'low': '2.8320', 'close': '2.8580', 'volume': 0, 'tradeTime': '20230117000000', 'tradeTimeinMills': 1673931600000}, {'open': '2.8570', 'high': '2.8970', 'low': '2.7950', 'close': '2.8160', 'volume': 0, 'tradeTime': '20230118000000', 'tradeTimeinMills': 1674018000000}, {'open': '2.8230', 'high': '2.8480', 'low': '2.8060', 'close': '2.8190', 'volume': 0, 'tradeTime': '20230119000000', 'tradeTimeinMills': 1674104400000}, {'open': '2.9190', 'high': '2.9540', 'low': '2.9050', '

In [20]:
json_data['barData']['priceBars']

[{'open': '2.9950',
  'high': '3.0400',
  'low': '2.9270',
  'close': '2.9990',
  'volume': 0,
  'tradeTime': '20230113000000',
  'tradeTimeinMills': 1673586000000},
 {'open': '2.9980',
  'high': '3.0410',
  'low': '2.9980',
  'close': '3.0150',
  'volume': 0,
  'tradeTime': '20230114000000',
  'tradeTimeinMills': 1673672400000},
 {'open': '3.0150',
  'high': '3.0460',
  'low': '2.9890',
  'close': '3.0030',
  'volume': 0,
  'tradeTime': '20230116000000',
  'tradeTimeinMills': 1673845200000},
 {'open': '2.9990',
  'high': '3.0590',
  'low': '2.8320',
  'close': '2.8580',
  'volume': 0,
  'tradeTime': '20230117000000',
  'tradeTimeinMills': 1673931600000},
 {'open': '2.8570',
  'high': '2.8970',
  'low': '2.7950',
  'close': '2.8160',
  'volume': 0,
  'tradeTime': '20230118000000',
  'tradeTimeinMills': 1674018000000},
 {'open': '2.8230',
  'high': '2.8480',
  'low': '2.8060',
  'close': '2.8190',
  'volume': 0,
  'tradeTime': '20230119000000',
  'tradeTimeinMills': 1674104400000},
 {'o

In [34]:
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['barData']['priceBars'] for d in json_data],
    parameters={"input_type": "passage", "truncate": "END"}
)

TypeError: string indices must be integers, not 'str'

In [44]:
processed_data =[]
client = OpenAI()

for barData in json_data['barData']['priceBars']:
    response = client.embeddings.create(
        input=float(barData["open"]),
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id":barData["tradeTime"],
        "metadata": {
            "ticker": "IT2Y",
            "date": barData["tradeTime"]
        }
    })

BadRequestError: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [42]:
embeddings = []
for record in json_data['barData']['priceBars']:
    # Convert numeric fields to float
    embedding = [
        float(record['open']),
        float(record['high']),
        float(record['low']),
        float(record['close']),
        record['tradeTime']  # Keep timestamp as is
    ]
    embeddings.append(embedding)

# Convert to numpy array for further processing (optional)


# Print the embeddings
print("Embeddings:")
print(embeddings)

Embeddings:
[[2.995, 3.04, 2.927, 2.999, '20230113000000'], [2.998, 3.041, 2.998, 3.015, '20230114000000'], [3.015, 3.046, 2.989, 3.003, '20230116000000'], [2.999, 3.059, 2.832, 2.858, '20230117000000'], [2.857, 2.897, 2.795, 2.816, '20230118000000'], [2.823, 2.848, 2.806, 2.819, '20230119000000'], [2.919, 2.954, 2.905, 2.94, '20230120000000'], [3.05, 3.098, 3.03, 3.094, '20230121000000'], [3.095, 3.123, 3.093, 3.102, '20230124000000'], [3.002, 3.02, 2.998, 3.008, '20230125000000'], [3.065, 3.095, 3.065, 3.083, '20230126000000'], [3.135, 3.152, 3.131, 3.149, '20230127000000'], [3.18, 3.203, 3.176, 3.194, '20230128000000'], [3.194, 3.299, 3.162, 3.294, '20230130000000'], [3.294, 3.392, 3.234, 3.24, '20230131000000'], [3.244, 3.314, 3.223, 3.288, '20230201000000'], [3.289, 3.303, 2.949, 2.976, '20230202000000'], [2.975, 3.061, 2.964, 3.032, '20230203000000'], [3.028, 3.065, 3.019, 3.056, '20230204000000'], [3.056, 3.165, 3.02, 3.146, '20230206000000'], [3.147, 3.21, 3.144, 3.192, '202302

In [43]:
index = pc.Index('anomaly')
index.upsert(
    vectors=embeddings,
    namespace="ticker-data"
)

ValueError: Invalid vector value passed: cannot interpret type <class 'list'>

In [61]:
vectors = []
for idx, record in enumerate(json_data['barData']['priceBars']):
    # Convert fields to float and create a vector
    embedding = [
        float(record['open']),
        float(record['high']),
        float(record['low']),
        float(record['close']),
        float(record['volume']),
        float(record['tradeTime'])  # Normalize timestamp for scaling
    ]
    vector = {
        "id": f"record-{idx}",
        "values": embedding,
        "metadata": {
            "ticker": "IT2Y",
             "open": float(record['open']),
              "date": record['tradeTime'][0:8] # Add tradeTime as metadata
        }
    }
    vectors.append(vector) # Unique identifier for each record

# Example: Upserting into Pinecone
# Assuming `pinecone-client` is installed and configured

# vectors = [(ids[i], embeddings[i]) for i in range(len(embeddings))]
# Initialize Pinecone
# pinecone.init(api_key="your_pinecone_api_key", environment="your_environment")
# index = pinecone.Index("your_index_name")



In [57]:
vectors

[{'id': 'record-0',
  'values': [2.995, 3.04, 2.927, 2.999, 0, '20230113000000'],
  'metadata': {'ticker': 'IT2Y', 'open': 2.995, 'date': '20230113'}},
 {'id': 'record-1',
  'values': [2.998, 3.041, 2.998, 3.015, 0, '20230114000000'],
  'metadata': {'ticker': 'IT2Y', 'open': 2.998, 'date': '20230114'}},
 {'id': 'record-2',
  'values': [3.015, 3.046, 2.989, 3.003, 0, '20230116000000'],
  'metadata': {'ticker': 'IT2Y', 'open': 3.015, 'date': '20230116'}},
 {'id': 'record-3',
  'values': [2.999, 3.059, 2.832, 2.858, 0, '20230117000000'],
  'metadata': {'ticker': 'IT2Y', 'open': 2.999, 'date': '20230117'}},
 {'id': 'record-4',
  'values': [2.857, 2.897, 2.795, 2.816, 0, '20230118000000'],
  'metadata': {'ticker': 'IT2Y', 'open': 2.857, 'date': '20230118'}},
 {'id': 'record-5',
  'values': [2.823, 2.848, 2.806, 2.819, 0, '20230119000000'],
  'metadata': {'ticker': 'IT2Y', 'open': 2.823, 'date': '20230119'}},
 {'id': 'record-6',
  'values': [2.919, 2.954, 2.905, 2.94, 0, '20230120000000'],
 

In [67]:
# Prepare vectors for upsert

index = pc.Index('anomaly2')
index.upsert(
    vectors=vectors,
    namespace="ticker-data"
)
# Upsert into Pinecone
index.upsert(vectors)

print(f"Upserted {len(vectors)} vectors into Pinecone.")

Upserted 620 vectors into Pinecone.
